# The Pandemic's Effect on Trends in Music: Exploration

Authors: Natnael Mekonnen, Sonya Lew, and Daniel Park

## Introduction

Why did we choose this topic? What is the importance?
Do we anticipate to be any changes?

## 1. Data Scraping

To explore the pandemic's effect on music, we chose to compare the COVID data in the US with the top 20 list on Spotify since January 2020. The COVID data will be extracted from the Atlantic's [The COVID Tracking Project](https://covidtracking.com/data/national) which has been constantly updating the data everyday with representatives from 50 states, 5 territories, and District of Colombia. On the other hand, the Spotify top 20 will be extracted from the [Spotify Charts](https://spotifycharts.com/) on a weekly starting from January 2020. The charts does not have a detail information on the song so we will get more detail by querying the Spotify API.  

Import the necessary libraries: 

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## 1.1 Load and View COVID data

In [67]:
covid_data = pd.read_csv('national-history.csv')
covid_data.head()

,date,death,deathIncrease,inIcuCumulative,inIcuCurrently,hospitalizedIncrease,hospitalizedCurrently,hospitalizedCumulative,negative,negativeIncrease,onVentilatorCumulative,onVentilatorCurrently,positive,positiveIncrease,recovered,states,totalTestResults,totalTestResultsIncrease
0,2020-12-07,274745.0,1347,32120.0,20098.0,3614,102148.0,589486.0,163378726,1386381,3328.0,7073.0,14717065,180193,5714557.0,56,205934174,1835388
1,2020-12-06,273398.0,1146,31946.0,20145.0,2311,101501.0,585872.0,161992345,1175503,3322.0,7095.0,14536872,177801,5624609.0,56,204098786,1648306
2,2020-12-05,272252.0,2461,31831.0,19947.0,3457,101192.0,583561.0,160816842,1530133,3321.0,7006.0,14359071,212880,5576152.0,56,202450480,2190899
3,2020-12-04,269791.0,2563,31608.0,19858.0,4652,101276.0,580104.0,159286709,1260657,3305.0,6999.0,14146191,224831,5470389.0,56,200259581,1854869
4,2020-12-03,267228.0,2706,31276.0,19723.0,5331,100755.0,575452.0,158026052,1238465,3280.0,6867.0,13921360,210204,5404018.0,56,198404712,1828230


The COVID data is detailed but since this project is finding its relationship with music, most of the unnecessary columns will be dropped.

In [68]:
covid_data = covid_data.drop(['death','deathIncrease','inIcuCumulative','inIcuCurrently', 'hospitalizedIncrease','hospitalizedCurrently','hospitalizedCumulative','onVentilatorCumulative','onVentilatorCurrently','recovered','states'], axis=1)

Knowing that the Spotify data is grouped on a weekly basis starting on a Friday, the next step is to prepare the COVID data so it is easy to merge.

In [69]:
# Creating a custom interval index for grouping the data which starts from begining of the year on a Friday and continues weekly
i = pd.to_datetime('01/02/2020')
bins = []
while i < pd.to_datetime('12/12/2020'):
    temp = i + pd.Timedelta('7 days')
    bins.append((i,temp))
    i = temp
bins = pd.IntervalIndex.from_tuples(bins)

In [70]:
# Convert the date from the dataframe to a pandas date and time format to be comuted in the cutting
covid_data['date'] = pd.to_datetime(covid_data['date'])

# Using the interval index created above, create a new column week which has the week interval of the data
covid_data['week'] = pd.cut(covid_data['date'], bins)

# Now that every row has a week interval, they will be grouped with the total for each column 
grouped_covid = covid_data.groupby(['week']).sum()

# Number the weeks to easily identify
grouped_covid['week#'] = list(range(1,len(bins)+1))

grouped_covid.head(10)

,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,week#
week,,,,,,,
"(2020-01-02, 2020-01-09]",0,0,0,0,0,0,1
"(2020-01-09, 2020-01-16]",0,0,0,0,0,0,2
"(2020-01-16, 2020-01-23]",0,0,0,0,3,1,3
"(2020-01-23, 2020-01-30]",0,0,0,0,22,3,4
"(2020-01-30, 2020-02-06]",0,0,0,0,81,11,5
"(2020-02-06, 2020-02-13]",0,0,0,0,134,6,6
"(2020-02-13, 2020-02-20]",0,0,0,0,205,14,7
"(2020-02-20, 2020-02-27]",0,0,0,0,6722,6451,8
"(2020-02-27, 2020-03-05]",4138,1943,953,367,50810,2376,9


In [71]:
column_names = ['week#','positiveIncrease','negativeIncrease','positive','negative','totalTestResultsIncrease','totalTestResults']
grouped_covid = grouped_covid.reindex(columns=column_names)

grouped_covid.head(10)

,week#,positiveIncrease,negativeIncrease,positive,negative,totalTestResultsIncrease,totalTestResults
week,,,,,,,
"(2020-01-02, 2020-01-09]",1,0,0,0,0,0,0
"(2020-01-09, 2020-01-16]",2,0,0,0,0,0,0
"(2020-01-16, 2020-01-23]",3,0,0,0,0,1,3
"(2020-01-23, 2020-01-30]",4,0,0,0,0,3,22
"(2020-01-30, 2020-02-06]",5,0,0,0,0,11,81
"(2020-02-06, 2020-02-13]",6,0,0,0,0,6,134
"(2020-02-13, 2020-02-20]",7,0,0,0,0,14,205
"(2020-02-20, 2020-02-27]",8,0,0,0,0,6451,6722
"(2020-02-27, 2020-03-05]",9,367,1943,953,4138,2376,50810


## 1.2 Load and View Spotify Top 20 Weekly Data

As mentioned above, the weekly Spotify top 10 list for 2020 has been downloaded as a csv file. However, since the data is not robust, the Spotify developer API was used to get more detail information on the songs and collectively stored under `data.csv`. The script to run this process is under `spotify_data_extraction.py`.

In [72]:
spotify_data = pd.read_csv('data.csv')
spotify_data.head()

,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,start_date,end_date
0,The Box,Please Excuse Me For Being Antisocial,Roddy Ricch,2019-12-06,196652,90,0.896,0.1040,0.896,0.586,0.00000,0.7900,-6.687,0.0559,116.971,4,2020-01-03,2020-01-10
1,ROXANNE,ROXANNE,Arizona Zervas,2019-10-10,163636,88,0.621,0.0522,0.621,0.601,0.00000,0.4600,-5.616,0.1480,116.735,5,2020-01-03,2020-01-10
2,Yummy,Yummy,Justin Bieber,2020-01-03,210426,79,0.687,0.3660,0.687,0.514,0.00000,0.1160,-6.612,0.0897,145.921,4,2020-01-03,2020-01-10
3,Circles,Hollywood's Bleeding,Post Malone,2019-09-06,215280,90,0.695,0.1920,0.695,0.762,0.00244,0.0863,-3.497,0.0395,120.042,4,2020-01-03,2020-01-10
4,BOP,KIRK,DaBaby,2019-09-27,159714,85,0.769,0.1890,0.769,0.787,0.00000,0.1290,-3.909,0.3670,126.770,4,2020-01-03,2020-01-10


In [73]:
spotify_data['start_date'] = pd.to_datetime(spotify_data['start_date'])

# Using the same interval index created for the covid_data, create a new column week which has the week interval
spotify_data['week'] = pd.cut(spotify_data['start_date'], bins)

# Now that week is included, we can reindex with only the columns we need
spotify_data = spotify_data.reindex(columns=['week','length', 'popularity',
       'danceability', 'acousticness', 'danceability.1', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'time_signature'])

spotify_data.head()

,week,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,"(2020-01-02, 2020-01-09]",196652,90,0.896,0.1040,0.896,0.586,0.00000,0.7900,-6.687,0.0559,116.971,4
1,"(2020-01-02, 2020-01-09]",163636,88,0.621,0.0522,0.621,0.601,0.00000,0.4600,-5.616,0.1480,116.735,5
2,"(2020-01-02, 2020-01-09]",210426,79,0.687,0.3660,0.687,0.514,0.00000,0.1160,-6.612,0.0897,145.921,4
3,"(2020-01-02, 2020-01-09]",215280,90,0.695,0.1920,0.695,0.762,0.00244,0.0863,-3.497,0.0395,120.042,4
4,"(2020-01-02, 2020-01-09]",159714,85,0.769,0.1890,0.769,0.787,0.00000,0.1290,-3.909,0.3670,126.770,4


In [74]:
# Now that every row has a week interval, they will be grouped with the mean for each column 
grouped_spotify = spotify_data.groupby(['week']).mean()
grouped_spotify.head()

,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
week,,,,,,,,,,,,
"(2020-01-02, 2020-01-09]",188656.35,75.55,0.71665,0.219063,0.71665,0.60050,0.033020,0.198585,-6.22025,0.116085,126.61265,3.90
"(2020-01-09, 2020-01-16]",208934.70,75.35,0.71400,0.265838,0.71400,0.55110,0.039695,0.179490,-7.12430,0.140355,128.23130,4.05
"(2020-01-16, 2020-01-23]",206069.10,77.70,0.72310,0.272378,0.72310,0.55853,0.007701,0.195490,-7.31150,0.158935,130.57085,4.05
"(2020-01-23, 2020-01-30]",204243.55,81.20,0.72620,0.265644,0.72620,0.56990,0.046191,0.185030,-7.03405,0.171040,133.48885,4.05
"(2020-01-30, 2020-02-06]",198553.85,80.50,0.73100,0.225643,0.73100,0.59270,0.032991,0.199450,-6.23020,0.170440,130.47640,4.05


## 1.3 Merge the Spotify and Covid Data

In [75]:
merged = grouped_covid.merge(grouped_spotify, left_on='week', right_on='week')
merged.head()

,week#,positiveIncrease,negativeIncrease,positive,negative,totalTestResultsIncrease,totalTestResults,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
week,,,,,,,,,,,,,,,,,,,
"(2020-01-02, 2020-01-09]",1,0,0,0,0,0,0,188656.35,75.55,0.71665,0.219063,0.71665,0.60050,0.033020,0.198585,-6.22025,0.116085,126.61265,3.90
"(2020-01-09, 2020-01-16]",2,0,0,0,0,0,0,208934.70,75.35,0.71400,0.265838,0.71400,0.55110,0.039695,0.179490,-7.12430,0.140355,128.23130,4.05
"(2020-01-16, 2020-01-23]",3,0,0,0,0,1,3,206069.10,77.70,0.72310,0.272378,0.72310,0.55853,0.007701,0.195490,-7.31150,0.158935,130.57085,4.05
"(2020-01-23, 2020-01-30]",4,0,0,0,0,3,22,204243.55,81.20,0.72620,0.265644,0.72620,0.56990,0.046191,0.185030,-7.03405,0.171040,133.48885,4.05
"(2020-01-30, 2020-02-06]",5,0,0,0,0,11,81,198553.85,80.50,0.73100,0.225643,0.73100,0.59270,0.032991,0.199450,-6.23020,0.170440,130.47640,4.05


## 2. Data Analysis and Visualization